In [102]:
import pandas as pd   
import warnings; warnings.simplefilter('ignore')
import numpy as np

# Game Theory

## Simplex Method

In [104]:
def convert_to_maximization_type(a,b,Coeff,m,n,objective):
    if objective != 'max':
        Coeff=[-Coeff[i] for i in range(n)]
    for i in range(m):
        if a[i][-1] != '<=':
            for j in range(n):
                a[i][j]=-a[i][j]
            b[i]=-b[i]
    return a,b,Coeff,m,n
                
                
    

In [105]:


#1
def initialize_simplex_table(a, b, m, n, Coeff,objective):
    convert_dict={'x'+str(i+1):float for i in range(n)}
    jconvert_dict={'x'+str(i+1):float for i in range(n)}
    a,b,Coeff,m,n=convert_to_maximization_type(a,b,Coeff,m,n,objective)
    Cj_dict={'x'+str(i+1):[Coeff[i]] for i in range(n)}
    Zj_dict={'x'+str(i+1):[0] for i in range(n)}
    for i in range(m):
        Cj_dict['s'+str(i+1)]=[0]
        Zj_dict['s'+str(i+1)]=[0]
        convert_dict['s'+str(i+1)]=float
        jconvert_dict['s'+str(i+1)]=float
    Cj_dict['sol']=0
    Zj_dict['sol']=0   
    convert_dict['sol']=float
    jconvert_dict['sol']=float
    #Cj_dict.extend({0 for i in range(m)])
    #Zj=[0 for i in range(m+n)]
    data = {'CBi':[0 for i in range(m)]} 
    convert_dict['CBi']=float
    data['basic_var']=['s'+str(i+1) for i in range(m)]
    convert_dict['basic_var']=str
    for i in range(n):
        data['x'+str(i+1)]=[a[j][i] for j in range(m)]
    #augmenting data with slack variables
    for i in range(m):
        data['s'+str(i+1)]=[float(i==j) for j in range(m)]
    data['sol']=[b[j] for j in range(m)]
    data['ratio']=[0 for j in range(m)]
    convert_dict['ratio']=float
    df = pd.DataFrame(data, index=[str(i+1) for i in range(m)]) 
    Zj = pd.DataFrame(Zj_dict, index=['Zj']) 
    Cj = pd.DataFrame(Cj_dict, index=['Cj'])
    
    df = df.astype(convert_dict) 
    
    Zj = Zj.astype(jconvert_dict)
    Cj = Cj.astype(jconvert_dict)
    return df,Cj,Zj
  

In [106]:
#2 
def print_table(df, Cj, Zj):
    print("simplex table:")
    frames = [df, Cj, Zj]
    result = pd.concat(frames)
    
    print(result)
#     print("Cj's: ",Cj)
    
#     print("Zj's: ",Zj)
    
    
  
 
# df,Cj,Zj = initialize_simplex_table(a, b, m, n, Coeff,objective)
# print_table(df,Cj,Zj)

In [107]:
#3
def Zj_Cj(Cj, Zj, m, n):
    l=['x'+str(i+1) for i in range(n)]
    l.extend(['s'+str(i+1) for i in range(m)])
    return  [Zj[i]['Zj']-Cj[i]['Cj'] for i in l]


In [108]:
#4
def col_name(i,n):
    if (i+1)<=n:
        return 'x'+str(i+1)
    else:
        return 's'+str(i+1-n)
    
def get_key_column(zj_cj, m, n):
    val_min=zj_cj[0]
    idx_min='x1' 
    #print("zj_cj",zj_cj)
    #print(m+n)
    for i in range(m+n):
        if val_min>zj_cj[i]:
            val_min=zj_cj[i]
            idx_min=col_name(i,n)
    
    return idx_min
    


def get_key_row(df, key_col, m):   
    df['ratio']=df['sol']/df[key_col]
    val_min=999999999
    idx_min=0
    for i in range(m):
        if val_min>df['ratio'][i]>=0:
            val_min=df['ratio'][i]
            idx_min=i
    return df, str(idx_min+1), val_min




In [110]:
def modify_table(key_row, key_col, Cj, Zj,df):
    df_new = df.copy(deep=True)
    Zj_new = Zj.copy(deep=True)
    col_len=len(df[key_col])
    row_len=len(df.loc[key_row])
    p=key_elem=df[key_col][key_row]
    entering_var=key_col
    leaving_var=df['basic_var'][key_row]
    print("Entering variable is ", entering_var)
    print("Leaving variable is ", leaving_var)
    df_new['basic_var'][key_row]=entering_var
    
    
    col=['x'+str(i+1) for i in range(n)]
    col.extend(['s'+str(i+1) for i in range(m)])
    col.append('sol')
    row=list(df.index)
    
    for j in col:
        r=df[j][key_row]              
        for i in row:    
            if j==key_col:
                df_new[j][i]=(-df[j][i])/p                
            elif i==key_row:
                df_new[j][i]=df[j][i]/p               
            else:
                s=df[j][i]
                q=df[key_col][i] 
                df_new[j][i]=(p*s-q*r)/p
    #print(1/p)
    df_new[key_col][key_row]=1/p
    #df_new[key_col]=df[]
    df_new[leaving_var]  = df_new[entering_var]    
    df_new[entering_var] = 0
    df_new[entering_var][key_row]=1
    
    #swap the coefficients value
    df_new['CBi'][key_row]=Cj[key_col]['Cj']
    
    #modify zj
    for j in col:
        Zj_new[j]['Zj']=sum([df_new[j][i]*df_new['CBi'][i] for i in row]) 
        #Zj_new[j]['Zj']=16.0*0.05
    return df_new,Cj,Zj_new
    
    
    
    
    
    
    
    
    

In [111]:
# df_new,Cj,Zj_new=modify_table(key_row, key_col, Cj, Zj,df)
# print_table(df_new,Cj,Zj_new)


In [112]:
#Check is optimal solution is reached or not
def is_optimal(zj_cj):
    val=min(zj_cj)
    if val>=0:
        print("Optimality is reached")
        return True
    else:
        print("Lets proceed to next iteration")
        return False

In [113]:
def simplex_method(a, b, m, n, Coeff,objective):    
    df,Cj,Zj = initialize_simplex_table(a, b, m, n, Coeff,objective)
    zj_cj=Zj_Cj(Cj, Zj, m,n)
    key_col=get_key_column(zj_cj, m, n)
    df, key_row, min_ratio = get_key_row(df, key_col, m)
    print_table(df,Cj,Zj)
    i=0
    val_max=0
    while not is_optimal(zj_cj) and i<10:
        print("_______________________________________________________")
        if df[key_col][key_row]==0:
            print("Problem is unbounded")
            return
        df_new,Cj,Zj_new=modify_table(key_row, key_col, Cj, Zj,df)
        
        df=df_new
        Zj=Zj_new
        
        if objective=="max":
            val_max=Zj['sol']['Zj']
            print("value of objective function is ",Zj['sol']['Zj'])
        else:
            val_max=-Zj['sol']['Zj']
            print("value of objective function is ",-Zj['sol']['Zj'])
        
        zj_cj=Zj_Cj(Cj, Zj,m,n)
        key_col=get_key_column(zj_cj, m, n)
        df, key_row, min_ratio = get_key_row(df, key_col, m)  
        print_table(df,Cj,Zj)
        i+=1
    l=['x'+str(i+1) for i in range(n)]
    l.extend(['s'+str(i+1) for i in range(m)])
    zj__cj= {i:(Zj[i]['Zj']-Cj[i]['Cj']) for i in l}
    return df, zj__cj, val_max
    
    

## Finding out saddle point

In [115]:
m=3
n=3
payoff=[[1,-1,3],
        [3,5,-3],
        [6,2,-2]]

In [116]:
def row_min_max(payoff,an):   
    payoff_np=np.array(payoff)
    min_val=np.array([np.amin(payoff_np[i,:]) for i in range(an)])
    max_val=np.amax(min_val)
    max_i=np.argmax(min_val)
    return max_i, max_val

def col_max_min(payoff,bn):
    payoff_np=np.array(payoff)
    max_val=np.array([np.amax(payoff_np[:,j]) for j in range(bn)])
    min_val=np.amin(max_val)
    min_i=np.argmin(max_val)
    return min_i, min_val
    
#if minmax==maxmin then the point is saddle pt

In [117]:
#make payoff matrix positive
def make_positive(payoff):
    payoff_np=np.array(payoff)
    min_val=np.amin(payoff_np)
    if min_val<0:
        payoff_np-=min_val
    else:
        min_val=0
    a=payoff_np.tolist()
    m=len(a)
    n=len(a[0])
    b=[1 for i in range(m)]
    for i in range(m):
        a[i].append("<=")
    Coeff=[1 for i in range(n)]
    return a, b, m, n, Coeff, "max",min_val
    

In [118]:
def optimal_values(df, zj__cj, val_max, min_val, m, n):
    v_=(1/val_max)
    v=v_+min_val
    yi=[0 for i in range(n)]
    xi=[0 for i in range(m)]    
    for i in range(m):
        val=df['basic_var'][i]
        if val[0]=='x':
            yi[int(val[1:])-1]=abs(df['sol'][i])*v_
        xi[i]=zj__cj["s"+str(i+1)]*v_
    print("--------------------------------------")
    print("Optimal Game value: ",v)
    print("Optimal stratedy of A: ", xi)
    print("Optimal stratedy of B: ", yi)
    
    

In [119]:
def game_theory(payoff):
    an=len(payoff)
    bn=len(payoff[0])
    aii,minmax=row_min_max(payoff,an)
    bjj,maxmin=col_max_min(payoff,bn)
    if maxmin==minmax:
        print("The solution has a saddle point.")
        print("The optimal game value occurs when player A uses strategy",aii+1, "and player B uses stratedy", bjj+1)
        print("The optimal game value is:",payoff[aii][bjj])
    else:
        a, b, m, n, Coeff, objective,min_val=make_positive(payoff)
        df, zj__cj, val_max=simplex_method(a, b, m, n, Coeff,objective)
        optimal_values(df, zj__cj, val_max, min_val, m, n)
        

In [120]:
#just for testing
game_theory(payoff)

simplex table:
    CBi basic_var   x1   x2   x3   s1   s2   s3  sol     ratio
1   0.0        s1  4.0  2.0  6.0  1.0  0.0  0.0  1.0  0.250000
2   0.0        s2  6.0  8.0  0.0  0.0  1.0  0.0  1.0  0.166667
3   0.0        s3  9.0  5.0  1.0  0.0  0.0  1.0  1.0  0.111111
Cj  NaN       NaN  1.0  1.0  1.0  0.0  0.0  0.0  0.0       NaN
Zj  NaN       NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0       NaN
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  s3
value of objective function is  0.1111111111111111
simplex table:
    CBi basic_var   x1        x2        x3   s1   s2        s3       sol  \
1   0.0        s1  0.0 -0.222222  5.555556  1.0  0.0 -0.444444  0.555556   
2   0.0        s2  0.0  4.666667 -0.666667  0.0  1.0 -0.666667  0.333333   
3   1.0        x1  1.0  0.555556  0.111111  0.0  0.0  0.111111  0.111111   
Cj  NaN       NaN  1.0  1.000000  1.000000  0.0  0.0  0.000000  0.000000   
Zj  NaN       NaN  1.0  0

In [121]:
m=n=2
payoff=[[5,2],
        [3,4]]
game_theory(payoff)

simplex table:
    CBi basic_var   x1   x2   s1   s2  sol     ratio
1   0.0        s1  5.0  2.0  1.0  0.0  1.0  0.200000
2   0.0        s2  3.0  4.0  0.0  1.0  1.0  0.333333
Cj  NaN       NaN  1.0  1.0  0.0  0.0  0.0       NaN
Zj  NaN       NaN  0.0  0.0  0.0  0.0  0.0       NaN
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  s1
value of objective function is  0.2
simplex table:
    CBi basic_var   x1   x2   s1   s2  sol     ratio
1   1.0        x1  1.0  0.4  0.2  0.0  0.2  0.500000
2   0.0        s2  0.0  2.8 -0.6  1.0  0.4  0.142857
Cj  NaN       NaN  1.0  1.0  0.0  0.0  0.0       NaN
Zj  NaN       NaN  1.0  0.4  0.2  0.0  0.2       NaN
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x2
Leaving variable is  s2
value of objective function is  0.2857142857142857
simplex table:
    CBi basic_var   x1   x2        s1        s2       sol     rati

In [122]:
m=2
n=4
payoff=[[-2,3,5,2],
        [5,-1,-2,0]]
game_theory(payoff)

simplex table:
    CBi basic_var   x1   x2   x3   x4   s1   s2  sol     ratio
1   0.0        s1  0.0  5.0  7.0  4.0  1.0  0.0  1.0       inf
2   0.0        s2  7.0  1.0  0.0  2.0  0.0  1.0  1.0  0.142857
Cj  NaN       NaN  1.0  1.0  1.0  1.0  0.0  0.0  0.0       NaN
Zj  NaN       NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0       NaN
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  s2
value of objective function is  0.14285714285714285
simplex table:
    CBi basic_var   x1        x2   x3        x4   s1        s2       sol  \
1   0.0        s1  0.0  5.000000  7.0  4.000000  1.0 -0.000000  1.000000   
2   1.0        x1  1.0  0.142857  0.0  0.285714  0.0  0.142857  0.142857   
Cj  NaN       NaN  1.0  1.000000  1.0  1.000000  0.0  0.000000  0.000000   
Zj  NaN       NaN  1.0  0.142857  0.0  0.285714  0.0  0.142857  0.142857   

       ratio  
1   0.142857  
2        inf  
Cj       NaN  
Zj       NaN  
Lets procee

In [123]:
m=2
n=4
payoff=[[4,0,6,-2],
        [2,6,1,7]]
game_theory(payoff)

simplex table:
    CBi basic_var   x1   x2   x3   x4   s1   s2  sol     ratio
1   0.0        s1  6.0  2.0  8.0  0.0  1.0  0.0  1.0  0.166667
2   0.0        s2  4.0  8.0  3.0  9.0  0.0  1.0  1.0  0.250000
Cj  NaN       NaN  1.0  1.0  1.0  1.0  0.0  0.0  0.0       NaN
Zj  NaN       NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0       NaN
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  s1
value of objective function is  0.16666666666666666
simplex table:
    CBi basic_var   x1        x2        x3   x4        s1   s2       sol  \
1   1.0        x1  1.0  0.333333  1.333333  0.0  0.166667  0.0  0.166667   
2   0.0        s2  0.0  6.666667 -2.333333  9.0 -0.666667  1.0  0.333333   
Cj  NaN       NaN  1.0  1.000000  1.000000  1.0  0.000000  0.0  0.000000   
Zj  NaN       NaN  1.0  0.333333  1.333333  0.0  0.166667  0.0  0.166667   

       ratio  
1        inf  
2   0.037037  
Cj       NaN  
Zj       NaN  
Lets procee

In [124]:
m=4
n=4
payoff=[[-1,0,4,-1],
        [5,1,3,1],
        [-2,0,-1,4],
        [-3,-2,4,5]]
game_theory(payoff)

The solution has a saddle point.
The optimal game value occurs when player A uses strategy 2 and player B uses stratedy 2
The optimal game value is: 1


#### Menu driven code

In [125]:
print("Enter number of strategies of A and B respectively")
m,n=map(int,input().split())
payoff=[]
print("Enter payoff matrix")
for i in range(m):
    listi=list(map(int,input().split()))
    payoff.append(listi)
game_theory(payoff)
    

Enter number of strategies of A and B respectively


 2 2


Enter payoff matrix


 5 2
 3 4


simplex table:
    CBi basic_var   x1   x2   s1   s2  sol     ratio
1   0.0        s1  5.0  2.0  1.0  0.0  1.0  0.200000
2   0.0        s2  3.0  4.0  0.0  1.0  1.0  0.333333
Cj  NaN       NaN  1.0  1.0  0.0  0.0  0.0       NaN
Zj  NaN       NaN  0.0  0.0  0.0  0.0  0.0       NaN
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  s1
value of objective function is  0.2
simplex table:
    CBi basic_var   x1   x2   s1   s2  sol     ratio
1   1.0        x1  1.0  0.4  0.2  0.0  0.2  0.500000
2   0.0        s2  0.0  2.8 -0.6  1.0  0.4  0.142857
Cj  NaN       NaN  1.0  1.0  0.0  0.0  0.0       NaN
Zj  NaN       NaN  1.0  0.4  0.2  0.0  0.2       NaN
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x2
Leaving variable is  s2
value of objective function is  0.2857142857142857
simplex table:
    CBi basic_var   x1   x2        s1        s2       sol     rati